In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

## Import Mobility Data

In [2]:
# keep FIPS as string to preserve leading zeroes

In [3]:
mob = pd.read_csv(r"/Users/philip.ballentine/Downloads/Region_Mobility_Report_CSVs/2020_US_Region_Mobility_Report.csv", dtype={'census_fips_code': 'str'} )

In [4]:
mob['date'] = mob['date'].astype('datetime64')

In [5]:
mob.dtypes

country_region_code                                           object
country_region                                                object
sub_region_1                                                  object
sub_region_2                                                  object
metro_area                                                   float64
iso_3166_2_code                                               object
census_fips_code                                              object
date                                                  datetime64[ns]
retail_and_recreation_percent_change_from_baseline           float64
grocery_and_pharmacy_percent_change_from_baseline            float64
parks_percent_change_from_baseline                           float64
transit_stations_percent_change_from_baseline                float64
workplaces_percent_change_from_baseline                      float64
residential_percent_change_from_baseline                     float64
dtype: object

## Join Full County-Level Dataset

In [6]:
counties = pd.read_csv(r"/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/counties_dataset_full.csv",dtype={'County_FIPS':'str'})

In [7]:
counties.dtypes

Unnamed: 0                 int64
County_FIPS               object
StateCD                   object
Name                      object
HS_DIPLOMA_ONLY_2018     float64
SOME_COLLEGE_2018        float64
BACHELORS_PLUS_2018      float64
Poverty_PCT_2018         float64
UrbanCD                    int64
Median_Income_2018       float64
Population_ACS           float64
Population_Black_ACS     float64
Population_Hisp_ACS      float64
Population_Native_ACS    float64
PCT_Black_ACS            float64
PCT_Hisp_ACS             float64
PCT_Native_ACS           float64
UrbanCD_Description       object
Metro                     object
dtype: object

In [8]:
counties['census_fips_code'] = counties['County_FIPS'].astype('str')

In [9]:
mobility_counties = mob.merge(counties, on = "census_fips_code")

In [10]:
mobility_counties.dtypes

country_region_code                                           object
country_region                                                object
sub_region_1                                                  object
sub_region_2                                                  object
metro_area                                                   float64
iso_3166_2_code                                               object
census_fips_code                                              object
date                                                  datetime64[ns]
retail_and_recreation_percent_change_from_baseline           float64
grocery_and_pharmacy_percent_change_from_baseline            float64
parks_percent_change_from_baseline                           float64
transit_stations_percent_change_from_baseline                float64
workplaces_percent_change_from_baseline                      float64
residential_percent_change_from_baseline                     float64
Unnamed: 0                        

In [11]:
mobility_counties.head(5)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,...,Median_Income_2018,Population_ACS,Population_Black_ACS,Population_Hisp_ACS,Population_Native_ACS,PCT_Black_ACS,PCT_Hisp_ACS,PCT_Native_ACS,UrbanCD_Description,Metro
0,US,United States,Alabama,Autauga County,NaN,NaN,01001,2020-02-15,5.0,7.0,...,59338.0,55036.0,10510.0,1467.0,169.0,19.096591,2.665528,0.307072,In small metro area of less than 1 million res...,Smaller_metro
1,US,United States,Alabama,Autauga County,NaN,NaN,01001,2020-02-16,0.0,1.0,...,59338.0,55036.0,10510.0,1467.0,169.0,19.096591,2.665528,0.307072,In small metro area of less than 1 million res...,Smaller_metro
2,US,United States,Alabama,Autauga County,NaN,NaN,01001,2020-02-17,8.0,0.0,...,59338.0,55036.0,10510.0,1467.0,169.0,19.096591,2.665528,0.307072,In small metro area of less than 1 million res...,Smaller_metro
3,US,United States,Alabama,Autauga County,NaN,NaN,01001,2020-02-18,-2.0,0.0,...,59338.0,55036.0,10510.0,1467.0,169.0,19.096591,2.665528,0.307072,In small metro area of less than 1 million res...,Smaller_metro
4,US,United States,Alabama,Autauga County,NaN,NaN,01001,2020-02-19,-2.0,0.0,...,59338.0,55036.0,10510.0,1467.0,169.0,19.096591,2.665528,0.307072,In small metro area of less than 1 million res...,Smaller_metro


## Bring in the COVID Data to be Joined In 

In [12]:
covid = pd.read_csv(r"/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/covid_dataset_full.csv",dtype={'County_FIPS':'str'})

In [13]:
covid.Date = covid.Date.astype("datetime64")

In [14]:
mobility_counties['Date'] = mobility_counties['date']



In [15]:
mobility_covid_urban = mobility_counties.merge(covid, on =['Date','County_FIPS'])

In [16]:
# Can't deal with the + signs, so remove these 

large_columns = list(mobility_covid_urban.columns)
large_columns = [x.replace('+','') for x in large_columns]
mobility_covid_urban.columns = large_columns

In [17]:
columns = [ 'date',
 'retail_and_recreation_percent_change_from_baseline',
 'grocery_and_pharmacy_percent_change_from_baseline',
 'parks_percent_change_from_baseline',
 'transit_stations_percent_change_from_baseline',
 'workplaces_percent_change_from_baseline',
 'residential_percent_change_from_baseline',
 'County_FIPS',
 'StateCD',
 'Name',
 'HS_DIPLOMA_ONLY_2018',
 'SOME_COLLEGE_2018',
 'BACHELORS_PLUS_2018',
 'Poverty_PCT_2018',
 'UrbanCD',
 'Median_Income_2018',
 'Population_ACS',
 'Population_Black_ACS',
 'Population_Hisp_ACS',
 'Population_Native_ACS',
 'PCT_Black_ACS',
 'PCT_Hisp_ACS',
 'PCT_Native_ACS',
 'UrbanCD_Description',
 'Metro',
 'Date',
 'State',
 'Confirmed',
 'Deaths',
 'Country/Region',
 'Deaths_Previous',
 'Confirmed_Previous',
 'Deaths_New',
 'Deaths_New_7',
 'Deaths_New_14',
 'Deaths_New_21',
 'Deaths_New_28',
 'Confirmed_New',
 'locationcol',
 'Confirmed_New_RollingAvg',
 'Deaths_New_7_RollingAvg',
 'Deaths_New_14_RollingAvg',
 'Deaths_New_21_RollingAvg',
 'Deaths_New_28_RollingAvg',
 ]

In [18]:
combined_dataset_sub = mobility_covid_urban[columns]

In [19]:
#check to see matching
combined_dataset_sub[['State','StateCD']].sample(frac=.00004)


,State,StateCD
419467,North Carolina,NC
292254,Minnesota,MN
93118,Georgia,GA
593422,Utah,UT
417725,North Carolina,NC
203899,Kansas,KS
183625,Iowa,IA
585188,Texas,TX
189442,Iowa,IA
548160,Texas,TX


In [27]:
list(samp.dtypes)

[dtype('<M8[ns]'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('<M8[ns]'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64')]

In [38]:
def create_percapita_measures(dataframe):
    list_col = list(dataframe.columns)
    for i in list_col:
        if dataframe[i].dtype == "O":
            #print("String")
            continue
        if "percent" in i or "PCT" in i:
            #print("PCT")
            continue
        if "death" in i.lower() or "confirmed" in i.lower() and "previous" not in i.lower() :
            try:
                new_col_name = i+'_PER_CAPITA'
                dataframe[new_col_name] = dataframe[i]/dataframe['Population_ACS']
            except:
                print("error occurred for {}") % i 

In [42]:
create_percapita_measures(combined_dataset_sub)

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


## Export to File

In [43]:
import os
os.chdir(r'/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/')
path = str(os.getcwd())
filename = "hia_covid_combined.csv"
combined_dataset_sub.to_csv(filename)
print("{filename} has been created in {path}".format(filename=filename, path=path))

hia_covid_combined.csv has been created in /Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets


In [44]:
import os
os.chdir(r'/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/')
path = str(os.getcwd())
filename = "hia_covid_combined_sample.csv"
combined_dataset_sub.sample(frac=.05).to_csv(filename)
print("{filename} has been created in {path}".format(filename=filename, path=path))

hia_covid_combined_sample.csv has been created in /Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets


In [45]:
# ratio to check join 
combined_dataset_sub.shape[0]/mobility_counties.shape[0]

0.9993187291472363